## Inversion for a coupled acoustic-gravity system

Before running `pip install tinyda ray' 

### Problem setup

Let $\varphi$ be the potential for the flow velocity, it is solution to 
\begin{align}\label{eq:forwardmodel}
    \frac{\partial^2 \varphi}{\partial t^2}
    - c^2 \Delta \varphi 
    + g \frac{\partial \varphi}{\partial z}
    = 0,  \quad \text{ in } \Omega,
    \\
    \frac{\partial^2 \varphi}{\partial t^2}
    + g \frac{\partial \varphi}{\partial z} 
    = 0, \quad \text{ on } \Gamma_s,
    \\
    \nabla \varphi \cdot {\bf n}
    = u_b, \quad \text{ on } \Gamma_b,
\end{align}
where $c$ is the (constant) sound speed, $g$ is the gravity acceleration, ${\bf n}$ is the outwards unit vector normal to the seabed $\Gamma_b$ and $u_b$ is the seabed velocity.
The fluid displacement $u$ is related to the potential by $u = \nabla \varphi$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from scipy.stats import multivariate_normal, uniform, norm, rv_continuous, wasserstein_distance
from scipy.spatial.distance import cdist
from scipy.spatial import distance_matrix

import pandas as pd
import tinyDA as tda
import umbridge

### Initialising UM-Bridge
Before running the following cell, the model Docker image must be started from a system terminal using

`docker run -it -p 4242:4242 acousticgravity`

The Dockerfile can be found in the acousticgracity branch on UM-Bridge benchmarks. If not just running on a local machine replace `localhost` with the server.

In [ ]:
# Create the config data
#### Change captor location and number
# Each element of the list "captors" is a tuple of (x,z) coordinates. 
# // version without PML: x should be between 6 and 36, z should be between 0 and (strictly less than) 1. 
# // version with PML: x should be between 1 and 29, z should be between 0 and (strictly less than) 1. 
captors = [(10,0.5), (18,0.9)] #[(28,0.5), (20,0.9)]

#### do not change this part
captors = np.array(captors, dtype=[('x',float),('y',float)])
captors = np.sort(captors, order=['x','y'])
fixedFloor= np.array([0,1]) # np.linspace(0,5,6)
Lx, Nx = 30, 150
Lz, Nz = 1, 10
for i in range(len(captors)):
    xCapt = captors[i][0]*Nx/Lx
    fixedFloor = np.concatenate((fixedFloor, [xCapt-2, xCapt-1, xCapt, xCapt+1, xCapt+2]))
fixedFloor=np.concatenate((fixedFloor, np.array([Nx-2,Nx-1])))  #np.concatenate((fixedFloor, np.linspace(37,42,6)))
fixedFloor=np.unique(fixedFloor)
fixedFloor=fixedFloor.tolist()
captors = captors.tolist()
print(fixedFloor)



def fill_input(_input, fixedFloor):
    full = np.zeros(Nx)
    counter = 0
    for i in range(len(full)):
        if i not in fixedFloor: 
            full[i] = _input[counter]
            counter +=1 
    return full

In [ ]:
# connect to the UM-Bridge model.
umbridge_model = umbridge.HTTPModel('http://localhost:4242', "forward")

# wrap the UM-Bridge model in the tinyDA UM-Bridge interface.
#config={"fixedFloor":[0,1,2,3,4,5,26,27,28,29,30,36,37,38,39,40,41,42]} 
config={"fixedFloor":fixedFloor, "captors":captors}
my_model = tda.UmBridgeModel(umbridge_model, umbridge_config=config)

In [ ]:
nx = umbridge_model.get_input_sizes(config)[0] #the input is the value of f(x) at each grid point
ny = umbridge_model.get_output_sizes(config)[0] #the outout is a time series of the pressure
print(f"input:{nx}, output:{ny}")

### Example of input creation

In [ ]:
length_scale = 5.0
eps = 1e-6
decay_rate = 10

x = np.linspace(0, nx - 1, nx).reshape(-1, 1)

def rbf_covariance(grid, length_scale, variance=1.0):
    dists = cdist(grid, grid, 'euclidean')
    return variance * np.exp(-0.5 * (dists / length_scale) ** 2)

cov = rbf_covariance(x, length_scale)
cov += eps * np.eye(nx)
mean = np.zeros(nx)
my_prior = multivariate_normal(mean=mean, cov=cov)

# def smooth_cut(x, x0, width, k):
#     step1 = 1 / (1 + np.exp(-k * (x - (x0 - width / 2))))
#     step2 = 1 / (1 + np.exp(-k * (x - (x0 + width / 2))))
#     return step1 - step2

# cut = smooth_cut(x.flatten(), nx / 2, nx * (1 - 14 / nx), decay_rate)

sample = my_prior.rvs()

def restriction(sampin):
    field_cut = sampin #* cut
    return (field_cut > eps).astype(int)

sample = restriction(sample)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
ax.set_xticks([i for i in range(0,nx,10)])
ax.grid()

ax.plot(sample,label='sample')
#ax.plot(cut,label='smooth cut')
#ax.plot(sample * cut, label='cut sample')
ax.legend()
ax.set_title("Binary Level Set Sample from Gaussian Random Field")
plt.show()

### Problem setup

In [ ]:
# generate synthetic data for input 0.5
# Input is a 15km wide fault centered around x
# Domain size is 150km
exact = np.zeros(nx)
# Should be between 0 and Nx
exact[60:80]=1
print(exact)
d_true = my_model(exact)

# add some noise to the model output
sigma_noise = 0.01
d = d_true + np.random.normal(loc=0, scale=sigma_noise, size=ny)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(24,2), sharex=True)
fig.suptitle('Input')
x=np.linspace(0,nx, nx)
t=np.linspace(0,25, int(ny/2))

d_true1 = np.array(d_true)[0:ny:2]
d_true2 = np.array(d_true)[1:ny:2]

ax[0].set_title('Captor 1')
ax[0].plot(t,d_true1)
ax[0].plot(t,d[::2])

ax[1].set_title('Captor 2')
ax[1].plot(t,d_true2)
ax[1].plot(t,d[1::2])

In [ ]:
# plot several output and the exact (with/without noise) input/output

# exact (with/without noise) input/output 
xCapt = np.array(captors)[:,0]
yCapt = np.array(captors)[:,1]
print(xCapt, yCapt)

# Plot exact input
figIn, axIn = plt.subplots(figsize=(24,4))

axIn.plot(xCapt,yCapt,'x')
exact_full = fill_input(exact, fixedFloor)
axIn.plot(np.linspace(0,Nx,Nx), exact_full)
#axIn.set_xticks([i for i in range(0,30)])
axIn.set_title("Exact inputs")


# fig, ax = plt.subplots(ncols=2, figsize=(24,4))
# t = np.linspace(0,10,int(ny/2))
# d_true1 = np.array(d_true)[0:ny:2]
# d_true2 = np.array(d_true)[1:ny:2]

# d1 = np.array(d)[0:ny:2]
# d2 = np.array(d)[1:ny:2]

# ax[0].set_title('Pressure, captor 1')
# ax[0].plot(t,d_true1, label='Noise-free input')
# ax[0].plot(t,d1, label='Noisy input')
# ax[0].legend()

# ax[1].set_title('Pressure, captor 2')
# ax[1].plot(t,d_true2, label='Noise-free input')
# ax[1].plot(t,d2, label='Noisy input')
# ax[1].legend()

In [ ]:
# plot several output and the exact (with/without noise) input/output
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(24,4))
figIn, axIn = plt.subplots(nrows=2, figsize=(24,4))

# exact (with/without noise) input/output 
xCapt = np.array(captors)[:,0]*Nx/Lx
yCapt = np.array(captors)[:,1]*Nz/Nz
print(xCapt, yCapt)

axIn[0].plot(xCapt,yCapt,'x')
exact_full = fill_input(exact, fixedFloor)
axIn[0].plot(exact_full)
axIn[0].set_title("Exact inputs")

t = np.linspace(0,10,int(ny/2))
d_true1 = np.array(d_true)[0:ny:2]
d_true2 = np.array(d_true)[1:ny:2]

d1 = np.array(d)[0:ny:2]
d2 = np.array(d)[1:ny:2]

ax[0][0].set_title('Pressure, captor 1')
ax[0][0].plot(t,d_true1, label='Noise-free input')
ax[0][0].plot(t,d1, label='Noisy input')
ax[0][0].legend()

ax[0][1].set_title('Pressure, captor 2')
ax[0][1].plot(t,d_true2, label='Noise-free input')
ax[0][1].plot(t,d2, label='Noisy input')
ax[0][1].legend()


# Plot several random samples
samples = []
inputs=[]
for x in range(3):
     log = my_prior.rvs()
     log = restriction(log)
     d_sample = my_model(log)

     d_sample1 = np.array(d_sample)[0:ny:2]
     d_sample2 = np.array(d_sample)[1:ny:2]
     
     ax[1][0].plot(t, d_sample1, label=f'samples {x}')
     ax[1][1].plot(t, d_sample2, label=f'samples {x}')

     log_full = fill_input(log,fixedFloor)
     axIn[1].plot(range(len(log_full)), log_full, label=f'input {x}')
     samples.append(d_sample)
     inputs.append(log)

ax[1][0].legend()
ax[1][1].legend()
axIn[1].set_title("Random inputs")
axIn[1].legend()

plt.show()

## Set up log-likelihood

Test standard Gaussian log-likelihood:
$$\log ⁡L(x)=-0.5(x-y)^T\Sigma^{-1}(x−y)$$

with a Wasserstein-2-based log-likelihood as in Dunlop+Yang, 2021:
$$\log ⁡L(x)\propto -\lambda W_2(x,y),$$

where:
- x = model output
- y = ground truth
- λ = scale parameter (e.g. inverse covariance)

While $W_1$ is contained in scipy.stats it seems $W_2$ is not

In [ ]:
class WassersteinLoglike:
    """
    Wasserstein-2 based log-likelihood
    """

    def __init__(self, data, lam=1.0):
        """
        Parameters
        ----------
        data : np.ndarray
            ground truth.
        lam : float
            Scaling factor (like inverse variance).
        """
        self.data = np.sort(data)
        self.lam = lam

    def loglike(self, x):
        """
        Compute the log-likelihood using squared W₂ distance.
        
        Parameters
        ----------
        x : np.ndarray
            Model output (same shape as data).
        
        Returns
        -------
        float
            Log-likelihood (unnormalized).
        """
        x_sorted = np.sort(x)
        return -self.lam * np.mean((x_sorted - self.data) ** 2)

    def grad_loglike(self, x):
        """
        Gradient of the W₂ log-likelihood.
        
        Returns
        -------
        np.ndarray
            Gradient of the log-likelihood.
        """
        n = len(x)
        sort_idx = np.argsort(x)
        x_sorted = x[sort_idx]
        y_sorted = self.data

        grad_sorted = -2 * self.lam * (x_sorted - y_sorted) / n

        grad = np.zeros_like(x)
        grad[sort_idx] = grad_sorted
        return grad
    
# Test for Gaussian log like
#sigma = 2.0
#cov_likelihood = sigma**2*np.eye(d_true.shape[0])
#my_loglike = tda.GaussianLogLike(d_true, cov_likelihood)

# Test for Wasserstain based log like
# TODO move implementation into tinyDA
my_loglike = WassersteinLoglike(d_true, 100.0)

In [ ]:
print("true ", np.exp(my_loglike.loglike(d_true)))
print("true + small noise ", np.exp(my_loglike.loglike(d)))

for d_sample in samples:
    print(np.exp(my_loglike.loglike(d_sample)))

In [ ]:
# initialise the Posterior
def levelset_model(parameters):
    levelset_params = restriction(parameters)
    return my_model(levelset_params)

my_posterior = tda.Posterior(my_prior, my_loglike, levelset_model)
#my_posterior = tda.Posterior(my_prior, my_loglike, my_model)

### Set up the proposal

In [ ]:
# Custom Crank-Nicolson with cutoff and levelset
class CrankNicolson(tda.CrankNicolson):
    def __init__(self, scaling, adaptive):
        super().__init__(scaling=scaling, adaptive=adaptive)

    def make_proposal(self, link):
        # make a pCN proposal.
        self.scaling = min(self.scaling, 1.0 - 1e-3)
        return np.sqrt(
            1 - self.scaling**2
        ) * link.parameters + self.scaling * np.random.multivariate_normal(
            self._mean, self.C
        )

# preconditioned Crank-Nicolson
pcn_scaling = 0.15
pcn_adaptive = True
my_proposal = CrankNicolson(scaling=pcn_scaling, adaptive=pcn_adaptive)

# # random walk Metropolis
#rwmh_cov = np.eye(nx)
#rmwh_scaling = 0.1
#rwmh_adaptive = False
#my_proposal = tda.GaussianRandomWalk(C=rwmh_cov, scaling=rmwh_scaling, adaptive=rwmh_adaptive)

### Sample

In [ ]:
# For testing purposes, iteration number is small for the given problem; Choose a larger number for real applications.
my_chains = tda.sample(my_posterior, my_proposal, iterations=6000, n_chains=2, force_sequential=True)

### Get some diagnostics

In [ ]:
import arviz as az

In [ ]:
# convert the tinyDA chains to an ArViz InferenceData object.
burnin = 1000
idata = tda.to_inference_data(my_chains, burnin=burnin)
az.to_netcdf(idata, "results_wassersteinlog.nc") # to store
idata = az.from_netcdf("results_wassersteinlog.nc")# to load

In [ ]:
print(idata)
print(idata.sample_stats)

In [ ]:
# display posterior summary statistics.
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", None)
info = az.summary(idata);
info

In [ ]:
# plot posterior kernel densities and traces.
az.plot_trace(idata)
plt.show()

In [ ]:
# extract the parameters from the chains.
subsampling_rate = 20
parameters = [link.parameters for link in my_chains['chain_0'][burnin::subsampling_rate] + my_chains['chain_1'][burnin::subsampling_rate]]

In [ ]:
n_samples = 100
ids = np.random.randint(0, len(parameters), n_samples)

# Evaluate model outputs and likelihoods
likelihoods = []
outputs = []

for i in ids:
    param = parameters[i]
    output = levelset_model(param)
    outputs.append(output)
    idata_sub = idata.sel(draw=slice(None, None, subsampling_rate))
    loglikes = idata_sub.sample_stats["likelihood"].values.flatten()
    #print("Likelihood ", loglikes[i])
    likelihoods.append(loglikes[i])

# Normalize likelihoods for colormap
likelihoods = np.array(likelihoods)
norm = colors.Normalize(vmin=np.min(likelihoods), vmax=np.max(likelihoods))
cmap = cm.viridis  # Or any matplotlib colormap
colors_list = [cmap(norm(l)) for l in likelihoods]

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 8))

d_true = np.array(d_true)
ax[0].plot(d_true[::2], 'k--', label="True output 1")
ax[1].plot(d_true[1::2], 'k--', label="True output 2")
for output in outputs:
    ax[0].plot(output[::2], alpha=0.03)
    ax[1].plot(output[1::2], alpha=0.03)
ax[0].legend()
ax[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
n_samples = 100

# Evaluate model outputs and likelihoods
likelihoods = []
outputs = []

for ids in range(n_samples):
    log = my_prior.rvs()
    log = restriction(log)
    output = levelset_model(log)
    outputs.append(output)
    idata_sub = idata.sel(draw=slice(None, None, subsampling_rate))
    loglikes = idata_sub.sample_stats["likelihood"].values.flatten()
    likelihoods.append(loglikes[i])

# Normalize likelihoods for colormap
likelihoods = np.array(likelihoods)
norm = colors.Normalize(vmin=np.min(likelihoods), vmax=np.max(likelihoods))
cmap = cm.viridis  # Or any matplotlib colormap
colors_list = [cmap(norm(l)) for l in likelihoods]

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 8))

d_true = np.array(d_true)
ax[0].plot(d_true[::2], 'k--', label="True output 1")
ax[1].plot(d_true[1::2], 'k--', label="True output 2")
for output in outputs:
    ax[0].plot(output[::2], alpha=0.03)
    ax[1].plot(output[1::2], alpha=0.03)
ax[0].legend()
ax[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 8))

all_links = my_chains['chain_0'][burnin::subsampling_rate] + my_chains['chain_1'][burnin::subsampling_rate]
sorted_links = sorted(all_links, key=lambda l: -l.likelihood)
top_links = sorted_links[:5]

d_true = np.array(d_true)
ax[0].plot(d_true[::2], 'k--', label="True output 1")
ax[1].plot(d_true[1::2], 'k--', label="True output 2")

for link in top_links:
    output = np.array(levelset_model(link.parameters))
    ax[0].plot(output[::2], alpha=0.3, label=f"loglike: {link.likelihood:.2f}")
    ax[1].plot(output[1::2], alpha=0.3, label=f"loglike: {link.likelihood:.2f}")
ax[0].legend()
ax[1].legend()
plt.tight_layout()
plt.show()


In [ ]:
x=np.linspace(0,nx, nx)
t=np.linspace(0,25, int(ny/2))
d1 = np.array(d_true)[0:ny:2]
d2 = np.array(d_true)[1:ny:2]

output = levelset_model(info['mean'])
o1 = np.array(output)[0:ny:2]
o2 = np.array(output)[1:ny:2]

fig, ax = plt.subplots(figsize=(12,2))
ax.plot(t,d1, label='True output 1')
ax.plot(t,o1, label='mean output 1')
ax.legend()

fig, ax = plt.subplots(figsize=(12,2))
ax.plot(t,d2, label='True output 2')
ax.plot(t,o2, '--' ,label='mean output 2')
ax.legend()

In [ ]:
x = range(len(info))
plt.figure(figsize=(8, 6))
plt.plot(exact)
plt.title('Exact input')
#plt.plot(x, restriction(info['mean']), 'o')  # x = index, y = mean
#plt.plot(x, info['mean'], 'x')  # x = index, y = mean
#print(top_links[0])
plt.plot(x, restriction(top_links[0].parameters), 'x')  # x = index, y = mean
plt.plot([captors[0][0]*Nx/Lx,captors[1][0]*Nx/Lx],[0,0],'x')
plt.ylabel('mean')
plt.title('Posterior Means')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
az.plot_autocorr(idata);

In [ ]:
az.plot_posterior(idata);

In [ ]:
az.plot_rank(idata);

## Sequential Monte Carlo

Adjustments:
- replace manual logging with arvis inference structure (easier/faster saving in netcdf format, immediate access to plotting routines)
- replace prior class with scipy priors (compatibility with tinyDA code above)
- replace loglikelihood class with tinyDA style loglikelihood
- replace MCMC steps with tinyDA

In [1]:
from os import sys, path, getcwd
sys.path.append(path.dirname(getcwd()))

import functools as ft

import warnings
import matplotlib.cbook
import scipy.stats as stats

from SMC import *

Finished: Mon Jul 28 19:00:22 2025


Start with the example provided before moving on to more complex model

In [2]:
### M:  number of particles/samples for SMC
num_samples = 1000

### var: uncertainty variance of measurements/modeling
std_noise = 0.05

### Title for saving the results
run = "acousticgravity"  # or any identifier you want
resultspath = f'RESULTS/SMC_M_{num_samples}_{run}'

# Wrapper for scipy prior
class prior:
    """
    Class for the prior distribution, here for uniform

    """
    def __init__(self, prior, d = 1):
        self.d = d # dimension of prior/parameter space, here 1
        self.RV = prior # this is specific to this prior

    def rvs(self, M):
        # Generate n samples. The resulting numpy array must be [1:d, 1:M]
        return np.array([self.RV.rvs(size=M) for i in range(self.d)])

    def logpdf(self, particle):
        # Compute logpdf at a point "particle"
        return np.sum([self.RV.logpdf(particle[i]) for i in range(self.d)])
    
a = 1.
b = 4.
my_prior = prior(stats.uniform(a,b))
    
### Set up forward model
def forward_model(x0):
    t_measurements = np.array([0.1,0.3,0.5,0.8, 1.]) # example: data are measurements at different times
    a = 1.    
    return x0*np.exp(-a*t_measurements)

def potential(particle, data, std_noise = 0.05):
    fwd_vals = forward_model(particle)
    return -1.*(np.linalg.norm(fwd_vals-data)**2/(2.*std_noise**2))

# Multiprocessing evaluation of previous potential 
def potential_mp(particles,data, std_noise = 0.05):
    pool = mp.Pool()
    potentials = pool.map(ft.partial(potential, data=data, std_noise=std_noise),particles.T)
    pool.close()
    return np.array(potentials)

### Generate synthetic data
prior_sample = my_prior.rvs(1)
print("Truth = ", prior_sample)
t_measurements = np.array([0.1,0.3,0.5,0.8, 1.]) # example: data are measurements at different times
forward_model_values = forward_model(prior_sample) 
data = forward_model_values + np.random.normal(0.,std_noise)
print("Noiseless data", forward_model_values)
print("Data", data)

### SET UP SMC SAMPLER  
SMC_sampler = SMC(my_prior, lambda x: potential_mp(x, data = data, std_noise = std_noise), resultspath, data, num_samples)
SMC_sampler.smc_tempering() ### perform SMC algorithm

Truth =  [[3.32142388]]
Noiseless data [[3.0053486  2.46057133 2.01454541 1.49241195 1.22188356]]
Data [[2.94114336 2.39636608 1.95034017 1.42820671 1.15767832]]
SMC for RESULTS/SMC_M_1000_acousticgravity - Started: Mon Jul 28 19:00:23 2025
------ TEMPERING-STEP 0 --------------------------

>> REWEIGHTED with exponent 0.0009765625
>> RESAMPLED (ESS 85.95% , exponent < 1)

--- duplicates: 344/1000  |  highest count: 2x
Applying MCMC kernel 4 times 

Acceptance ratio: 0.675 >0.3 
  => variance scaling: 1.0->2.0
>> MCMC complete!
--- duplicates: 0/1000  |  highest count: 1x
[saved intermediate result]
Tempering steps done:  0
------ TEMPERING-STEP 1 --------------------------

>> REWEIGHTED with exponent 0.0039033889770507812
>> RESAMPLED (ESS 72.3% , exponent < 1)

--- duplicates: 572/1000  |  highest count: 2x
Applying MCMC kernel 4 times 

Acceptance ratio: 0.2805 ~ 0.25 
  => keep variance scaling: 2.0
>> MCMC complete!
--- duplicates: 52/1000  |  highest count: 2x
[saved intermediat